In [ ]:
import pandas as pd

def merge(dataframe_list):
    """
            Use pandas to concatenate two dataframes, then drop duplicate rows 
            and finally sort by start time
    """
    m=pd.concat(dataframe_list)
    m=m.drop_duplicates(subset=['start time'], keep='first')
    m=m.sort_values(by=['start time'],ascending=True)
    return m

def find_multiple(path):
    from collections import Counter
    '''
    finds those summary files in the path that are not posid-unique
    Input: 
        path: path to check
    Returns: 
        not_unique: dictionary with positioner IDs that are not unique (i.e. multiple files exist) and 
                    list of files
    Example:'/Volumes/Saturn/data/summaries/M03691_summary.csv'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv.mine'
            '/Volumes/Saturn/data/summaries/M03692_summary.csv.r115418'
            '/Volumes/Saturn/data/summaries/M03693_summary.csv'
            '/Volumes/Saturn/data/summaries/M03694_summary.csv'
    
        The dictionary {'M03692':['/Volumes/Saturn/data/summaries/M03692_summary.csv',
        '/Volumes/Saturn/data/summaries/M03692_summary.csv.mine',
        '/Volumes/Saturn/data/summaries/M03692_summary.csv.r115418]] will be returned.
    '''
    not_unique={}
    file_list=glob.glob(path+'M*.csv*')
    posids=[f.lstrip(path)[:6] for f in file_list]
    not_unique_list=[k for (k,v) in Counter(posids).items() if v > 1]
    for n in not_unique_list:
        files=[k for k in file_list if k[:len(path)+6]==path+n]
        not_unique[n]=files
    return not_unique

# main
# Find and merge summary files with identical positioner ID (in the same path)
# Those files are created due to problems with SVN committ / update
# See function 'find_multiple' for an example
# Adjust the path below to fit your needs.
# The merged summary file will have a filename ending in '_summary_merged.csv'
# The original files are kept intact.

VERBOSE=True
path='/Volumes/Saturn/data/summaries/'
not_unique=find_multiple(path)
if VERBOSE: print(not_unique.keys())

for k in not_unique.keys():
    if VERBOSE: print('posid: '+str(k))
    file_list=not_unique[k]
    if VERBOSE:
        for f in file_list: print(f)
    df_list=[pd.read_csv(f) for f in file_list]
    m=merge(df_list)
    new_filename=file_list[0][0:file_list[0].find(k)]+k+'_summary_merged.csv'
    if VERBOSE: print(new_filename)
    m.to_csv(new_filename)

